In [ ]:
!pip install gensim

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
path = '/content/drive/MyDrive'

file4 = path + '/train.txt'
file5 = path + '/test.txt'

In [ ]:
import pandas as pd
import re

file3 = path+'/records.tsv'

df = pd.read_csv(file3, sep="\t", header=None, names=['word', 'stemm', 'lemm'])
words = list(df['word'])

In [ ]:
def split_into_sentences_init(text):

    sents = []

    text0 = re.sub('\n', ' ', text)
    text0 = re.sub('\s$|\"', '', text0)

    text1 = re.split('\s{2,}', text0)
    sents.append(text1[0]+'.')

    text2 = ' '.join(text1[1:])

    text3 = re.split('(?<=[.!?]) +', text2)

    text3 = [t for t in text3 if t != '']

    sents += text3



    return sents

def split_into_words_init(sents):

    words = []

    for s in sents:

        s0 = re.sub('xc2xa', '', s)

        s0 = re.sub(' ', '__probel__', s0)

        s0 = re.sub('(?<=(Dr|Mr|Ms))(__probel__)(?=[A-Z])', ' ', s0)


        w = re.split('__probel__', s0)
        w = [w_i for w_i in w if w_i != '' and w_i != '-']
        words.append(w)




    return words


In [ ]:
def split_into_sentences(text):


    sents = []
    for txt in text:
        txt_s = []

        text0 = re.sub('\n', ' ', txt)
        text0 = re.sub('\s$|\"', '', text0)

        text1 = re.split('\s{2,}', text0)
        txt_s.append(text1[0]+'.')

        text2 = ' '.join(text1[1:])

        text3 = re.split('(?<=[.!?]) +', text2)

        text3 = [t for t in text3 if t != '']

        txt_s += text3

        sents.append(txt_s)

    return sents

def split_into_words(texts):

    words = []

    for txt in texts:
        sent_words = []
        for s in txt:

            s0 = re.sub('xc2xa', '', s)

            s0 = re.sub(' ', '__probel__', s0)

            s0 = re.sub('(?<=(Dr|Mr|Ms))(__probel__)(?=[A-Z])', ' ', s0)


            w = re.split('__probel__', s0)
            w = [re.sub('[\.{1,}!?,:;\(\)\"\[\]\'\s]', '', str(w_i)).lower() for w_i in w if w_i != '' and w_i != '-']

            sent_words.append(w)

        words.append(sent_words)


    return words


In [ ]:
train = open(file4)
s_tr = train.read()

test = open(file5)
s_tst = test.read()

s_trn_spl = re.split('\_{5}', s_tr)
s_tst_spl = re.split('\_{5}', s_tst)

In [ ]:
train_sents = split_into_sentences_init(s_tr)
train_w = split_into_words_init(train_sents)

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
import gensim
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action='ignore')


model = gensim.models.Word2Vec(train_w, min_count=5,
                     window=7,
                     vector_size=800)


In [ ]:
def cosim(a, b):
    a_v = np.array(model.wv[a])
    b_v = np.array(model.wv[b])

    return 1-np.dot(a_v, b_v)/(norm(a_v)*norm(b_v))

In [ ]:
print(cosim('performer', 'singer'))
print(cosim('performer', 'musician'))

print()
print(cosim('performer', 'art'))
print(cosim('performer', 'show'))
print()
print(cosim('performer', 'users'))
print(cosim('performer', 'government'))

In [ ]:
print(cosim('mobile', 'phone'))
print(cosim('device', 'phone'))
print()
print(cosim('phone', 'game'))
print(cosim('phone', 'number'))
print()
print(cosim('phone', 'car'))
print(cosim('phone', 'month'))

In [ ]:
def vectorize_text(tx):
    sents = split_into_sentences(tx)
    w = split_into_words(sents)


    embs = []
    for el in w: #text lvl
        l_el1 = []
        for el2 in el: #sents lvl
            l_el2 = []
            for el3 in el2: #words lvl

                try:
                    l_el2.append(model.wv[el3])
                except KeyError:
                    pass
            if len(l_el2) > 0:
                l_el1.append(l_el2)
        if len(l_el1) > 0:
            embs.append(l_el1)


    s_m = []
    for el1 in embs:
        s_m2 = [] #mean (text) by sent
        for el2 in el1:
            s_m3 = [] #mean (sent) by words
            for el3 in el2:
                s_m3.append(el3)
            s_m2.append(sum(s_m3) / len(s_m3))
        s_m.append(sum(s_m2) / len(s_m2))

    return s_m, s_m2




In [ ]:
s_m, s_m2 = vectorize_text(s_trn_spl) #train

In [ ]:
s_m_t, s_m2_t = vectorize_text(s_tst_spl) #test

In [ ]:
import csv
import os

file = path+'/train_lab3.tsv'

try:
    os.remove(file)
except OSError:
    pass



with open(file, 'w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')

        for i, embed in enumerate(s_m):
            writer.writerow([i] + list(embed))

        size = tsvfile.truncate()
        size = tsvfile.truncate(size - 2)



In [ ]:
file = path+'/test_lab3.tsv'

try:
    os.remove(file)
except OSError:
    pass



with open(file, 'w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')

        for i, embed in enumerate(s_m_t):
            writer.writerow([i] + list(embed))

        size = tsvfile.truncate()
        size = tsvfile.truncate(size - 2)